Topic: Django Signals

Question 1 : By default are django signals executed synchronously or asynchronously? 
           Please support your answer with a code snippet that conclusively proves your stance.
           The code does not need to be elegant and production ready, we just need to understand your logic .

Answer : By default, Django signals are executed synchronously. This means that when a signal is sent, the code that handles the signal is executed immediately and blocks the execution of subsequent code until the signal handler finishes.

In [ ]:
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.http import HttpResponse
from .models import MyModel
import time

# Signal handler
@receiver(post_save, sender=MyModel)
def my_signal_handler(sender, instance, **kwargs):
    time.sleep(3)  # Simulate delay
    print("Signal handled.")

# View triggering the signal
def my_view(request):
    MyModel.objects.create(name="Test")
    return HttpResponse("Signal triggered, response delayed if synchronous.")


Question 2 : Do django signals run in the same thread as the caller? 
             Please support your answer with a code snippet that conclusively proves your stance.
             The code does not need to be elegant and production ready, we just need to understand your logic.


Answer : Yes, Django signals run in the same thread as the caller by default. This means when a signal is triggered, it is handled in the same execution thread as the function that sent the signal.

In [ ]:
import threading
from django.db.models.signals import post_save
from django.dispatch import receiver
from .models import MyModel
from django.http import HttpResponse

# Signal handler
@receiver(post_save, sender=MyModel)
def my_signal_handler(sender, instance, **kwargs):
    print(f"Signal handled in thread: {threading.current_thread().name}")

# View triggering the signal
def my_view(request):
    print(f"View running in thread: {threading.current_thread().name}")
    MyModel.objects.create(name="Test")
    return HttpResponse("Check console for thread info.")

Question 3 : By default do django signals run in the same database transaction as the caller?
             Please support your answer with a code snippet that conclusively proves your stance.
             The code does not need to be elegant and production ready, we just need to understand your logic.


Answer : By default, Django signals do not necessarily run in the same database transaction as the caller. This depends on when the signal is sent relative to the transaction lifecycle. For example, the post_save signal is sent after the object is saved to the database, but if it's part of a larger transaction (e.g., inside an atomic() block), the signal may run before the entire transaction is committed.

In [ ]:
from django.db import transaction
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.http import HttpResponse
from .models import MyModel

# Signal handler
@receiver(post_save, sender=MyModel)
def my_signal_handler(sender, instance, **kwargs):
    if transaction.get_connection().in_atomic_block:
        print("Signal handler: Inside transaction.")
    else:
        print("Signal handler: Outside transaction.")

# View triggering the signal
def my_view(request):
    with transaction.atomic():  # Start a transaction
        MyModel.objects.create(name="Test")
        print("View: Object created inside transaction.")

    return HttpResponse("Check console for transaction info.")

Topic: Custom Classes in Python

Description: You are tasked with creating a Rectangle class with the following requirements:

An instance of the Rectangle class requires length:int and width:int to be initialized.
We can iterate over an instance of the Rectangle class 
When an instance of the Rectangle class is iterated over, we first get its length in the format: {'length': <VALUE_OF_LENGTH>} followed by the width {width: <VALUE_OF_WIDTH>}

In [ ]:
class Rectangle:
    def __init__(self, length: int, width: int):
        self.length = length
        self.width = width
    
    # This method makes the class iterable
    def __iter__(self):
        # We yield the length and width in the required format
        yield {'length': self.length}
        yield {'width': self.width}

# Example usage
rect = Rectangle(5, 10)

# Iterating over the instance
for dimension in rect:
    print(dimension)

Explanation:
* The __iter__ method is defined to make the class iterable. It uses yield to return each dimension (length first, then width) in the specified format.
* When you iterate over an instance of Rectangle, you'll get the length first as {'length': <value>}, followed by the width as {'width': <value>}.